In [1]:
!pip install fastapi sqlalchemy databases uvicorn nest_asyncio

                                              0.0/94.7 kB ? eta -:--:--
                                              0.0/94.7 kB ? eta -:--:--
                                              0.0/94.7 kB ? eta -:--:--
     ----                                     10.2/94.7 kB ? eta -:--:--
     ----------------                       41.0/94.7 kB 653.6 kB/s eta 0:00:01
     ------------------------               61.4/94.7 kB 544.7 kB/s eta 0:00:01
     ------------------------------------   92.2/94.7 kB 581.0 kB/s eta 0:00:01
     -------------------------------------- 94.7/94.7 kB 451.8 kB/s eta 0:00:00
                                              0.0/63.7 kB ? eta -:--:--
     -------------------------                41.0/63.7 kB ? eta -:--:--
     -------------------------                41.0/63.7 kB ? eta -:--:--
     -------------------------                41.0/63.7 kB ? eta -:--:--
     -------------------------                41.0/63.7 kB ? eta -:--:--
     ----------------------

In [2]:
# database.py setup in Jupyter

from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./test.db"  # Uses an SQLite database file for simplicity

engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
metadata = MetaData()


In [4]:
# models.py setup in Jupyter

from sqlalchemy import Column, Integer, String, DateTime, Enum, ForeignKey
from sqlalchemy.orm import declarative_base
import datetime

Base = declarative_base()

class Contact(Base):
    __tablename__ = "contacts"
    
    id = Column(Integer, primary_key=True, index=True)
    phoneNumber = Column(String, nullable=True)
    email = Column(String, nullable=True)
    linkedId = Column(Integer, ForeignKey('contacts.id'), nullable=True)  # Points to another contact if linked
    linkPrecedence = Column(Enum("primary", "secondary", name="link_precedence"), default="primary")
    createdAt = Column(DateTime, default=datetime.datetime.utcnow)
    updatedAt = Column(DateTime, default=datetime.datetime.utcnow, onupdate=datetime.datetime.utcnow)
    deletedAt = Column(DateTime, nullable=True)

# Create the tables
Base.metadata.create_all(bind=engine)


In [5]:
# services.py logic in Jupyter

from sqlalchemy.orm import Session
from sqlalchemy import or_

def find_or_create_contact(db: Session, email: str, phone_number: str):
    # Fetch contacts with matching email or phone number
    contacts = db.query(Contact).filter(
        or_(Contact.email == email, Contact.phoneNumber == phone_number)
    ).all()

    if contacts:
        primary_contact = next((contact for contact in contacts if contact.linkPrecedence == "primary"), contacts[0])
        emails = {c.email for c in contacts if c.email}
        phone_numbers = {c.phoneNumber for c in contacts if c.phoneNumber}
        secondary_ids = [c.id for c in contacts if c.linkPrecedence == "secondary"]

        if email not in emails or phone_number not in phone_numbers:
            new_contact = Contact(
                email=email,
                phoneNumber=phone_number,
                linkedId=primary_contact.id,
                linkPrecedence="secondary"
            )
            db.add(new_contact)
            db.commit()
            db.refresh(new_contact)
            secondary_ids.append(new_contact.id)
            emails.add(email)
            phone_numbers.add(phone_number)
        
        return {
            "primaryContactId": primary_contact.id,
            "emails": list(emails),
            "phoneNumbers": list(phone_numbers),
            "secondaryContactIds": secondary_ids
        }
    else:
        new_contact = Contact(email=email, phoneNumber=phone_number, linkPrecedence="primary")
        db.add(new_contact)
        db.commit()
        db.refresh(new_contact)
        
        return {
            "primaryContactId": new_contact.id,
            "emails": [new_contact.email],
            "phoneNumbers": [new_contact.phoneNumber],
            "secondaryContactIds": []
        }


In [6]:
# app.py setup in Jupyter

from fastapi import FastAPI, Depends, HTTPException
import nest_asyncio
from sqlalchemy.orm import Session

# Apply nest_asyncio to allow FastAPI to run in Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Dependency to get DB session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

@app.post("/identify")
async def identify_contact(email: str, phone_number: str, db: Session = Depends(get_db)):
    try:
        result = find_or_create_contact(db, email, phone_number)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail="An unexpected error occurred")


In [7]:
import uvicorn

# Start the Uvicorn server
uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [17204]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57696 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51442 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17204]


In [9]:
curl -X POST "http://127.0.0.1:8000/identify" -H "Content-Type: application/json" -d '{"email": "test@example.com", "phone_number": "1234567890"}'


SyntaxError: invalid syntax (991857316.py, line 1)